In [1]:
from focus_stack.stack_framework import *
from focus_stack.align import AlignLayers

In [2]:
job = StackJob("job", "E:/Focus stacking/2024-03-17 - Baltic Diptera/A/")
job.add_action(AlignLayers(job.working_directory, "align", input_path="Immagini modificate", method=AlignLayers.ALIGN_RIGID))
#job.add_action(BalanceLayers(job.working_directory, "balance", input_path="align", method=BalanceLayers.BALANCE_LUMI, mask_radius=0.8, i_min=10, i_max=255))
#job.add_action(FocusStackBunch(job.working_directory, "batches", input_path="balance", exif_dir="Immagini modificate", postfix='_stack_pyr', denoise=0.8, method=FocusStackBunch.METHOD_PYRAMID, energy=FocusStackBunch.ENERGY_LAPLACIAN))
#job.add_action(FocusStack(job.working_directory, "stack", input_path="batches", exif_dir="Immagini modificate", postfix='_stack_pyr', denoise=0.8, method=FocusStack.METHOD_PYRAMID, energy=FocusStack.ENERGY_LAPLACIAN))

In [ ]:
job.run()

running align
65 files in folder: 'E:/Focus stacking/2024-03-17 - Baltic Diptera/A/Immagini modificate'


align:   0%|          | 0/65 [00:00<?, ?it/s]